In [1]:
#encoding:utf-8
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time

from BeautifulSoup import *
from urlparse import urljoin
import sys
import chardet
import re
from sqlite3 import dbapi2 as sqlite
import urllib2
import sys  
from PyQt4.QtGui import *  
from PyQt4.QtCore import *  
from PyQt4.QtWebKit import *  
from lxml import html 
import PyQt4

import random
import pandas as pd
import numpy as np
import os.path
import pylab
import matplotlib.pyplot as plt
import re

import threading
import pp
from multiprocessing import Pool

##load hk sg

In [2]:
monitor_HK=pd.read_csv('remainTTL'+'HK'+'.csv',
                       sep=' ',
                       header=None,
                       names=['domain','ttl','time'])
monitor_HK=monitor_HK.join(pd.Series(['211.136.20.203']*len(monitor_HK),name='dns'))

In [3]:
monitor_SG=pd.read_csv('remainTTL'+'SG'+'.csv',
                       sep=' ',
                       header=None,
                       names=['domain','ttl','time'])
monitor_SG=monitor_SG.join(pd.Series(['333.333.333.333']*len(monitor_SG),name='dns'))

In [18]:
HK_domain=monitor_HK[monitor_HK.ttl!=0].domain.drop_duplicates()

11                      0.gravatar.com
64                      0.pool.ntp.org
6928                 1.bp.blogspot.com
6985                    1.pool.ntp.org
8992                           163.com
10491                2.bp.blogspot.com
10523                   2.pool.ntp.org
11193                244d80.r.axf8.net
11884                3.bp.blogspot.com
12874                4.bp.blogspot.com
13078                         4399.com
13939                           56.com
15754                 801.tianyaui.com
15779                    806.tianya.cn
16572                     947.oadz.com
18306                    a-m-s.poco.cn
18364            a.cdn.intentmedia.net
18372                        a.cntv.cn
18406         a.freewebhostingarea.com
18580                     a.rfihub.com
18596                       a.tbcdn.cn
18599               a.tribalfusion.com
18603                     a.youdao.com
18657                  a1.mediagra.com
18728           a1331.phobos.apple.com
18797           a1946.pho

In [26]:
s=pd.Series([0]*len(HK_domain),index=HK_domain)

In [29]:
s.drop(befiltered).index

Index([u'0.pool.ntp.org', u'1.pool.ntp.org', u'2.pool.ntp.org',
       u'244d80.r.axf8.net', u'801.tianyaui.com', u'947.oadz.com',
       u'a.freewebhostingarea.com', u'a.rfihub.com', u'a.tbcdn.cn',
       u'a.tribalfusion.com', 
       ...
       u'x.co', u'ns1.satcom.it', u'nt1.duomiyy.com', u'nt2.duomiyy.com',
       u'www.jj.cn', u'ps.duomiyy.com', u'avkoo.best-adult-site.net',
       u'res.pplive.com', u'resource.weiphone.com', u'www.jizzle.com'],
      dtype='object', name=u'domain', length=4159)

In [30]:
pd.DataFrame(s.drop(befiltered).index).to_csv('HKdomain.txt',index=None,header=None)

###  load maxTTL

In [4]:
maxTTL=pd.read_csv('maxTTL.txt',
                  sep=' ',
                  header=None,
                  names=['maxttl'],
                  index_col=0).maxttl

let the malurl have max TTL

In [9]:
a=open('malurl_filtered10000.txt')
b=[]
for i in a.readlines():
    b.append(i.strip())

In [16]:
pd.DataFrame(maxTTL[b]).to_csv('maxTTL_filtered10k.txt',index=True,sep=' ',header=False)

### combine data

In [3]:
monitorCnt=0
monitor=pd.DataFrame(
pd.read_csv('remainTTL.csv',
            sep=',',
            header=None,
            names=['domain','dns','ttl','time']))
#monitor=monitor.join( pd.Series([1]*len(monitor),name='cnt'))
monitor=monitor.append(monitor_SG.append(monitor_HK,ignore_index=True),ignore_index=True)
hitrate=pd.Series([1]*len(monitor))
hitrate.name='hitrate'
hitrate[monitor.ttl==0]=0
monitor=monitor.join(hitrate)

NameError: name 'monitor_SG' is not defined

######  drop not morph ttl

In [229]:
weirddomain=[]
for i in monitor.groupby(monitor.domain):
    idx=i[1].ttl>maxTTL[i[0]]
    weirddomain.extend(i[1].index[idx])    

In [232]:
maxTTL[monitor.domain[weirddomain].drop_duplicates()].to_csv('weirddomain.txt',index=True)

In [44]:
monitor=monitor.drop(alldomain).reset_index(drop=True)

NameError: name 'alldomain' is not defined

##### make pivot

In [6]:
monitor_Gb=monitor['hitrate'].groupby([monitor['dns'],monitor['domain']]).sum()
monitorp=monitor_Gb.unstack()
monitorp=monitorp.fillna(0)
monitorp[monitorp!=0]=1

### load filter

In [14]:
Domainfilter=pd.read_csv('top-1m.csv',
                       sep=',',
                       header=None,
                       names=['idx','domain'])
prefixed='.'+Domainfilter.domain

In [15]:
def matchList(s):
    TLD=s.rfind('.')
    matchlist=[]
    if TLD!=-1:
        idx=TLD
        while True:
            idx=s.rfind('.',0,idx)
            matchlist.append(s[idx+1:])
            if idx==-1:
                break
    return matchlist

In [16]:
groupFilter=Domainfilter.domain[:10000]
length=[]
for i in groupFilter:
    length.append(len(i))
groupFilter=groupFilter.groupby(pd.Series(length))
hasKey=set()
for i in range(0,100):
    if groupFilter.groups.has_key(i)==True:
        hasKey.add(i)

In [17]:
def scanDomain(domain):
    global befiltered,hasKey,groupFilter
    for subdomain in matchList(domain):
        key=len(subdomain)
        if key in hasKey:
            if len((groupFilter.get_group(key)==subdomain).nonzero()[0]) !=0:
                befiltered.append(domain)
                break

In [11]:
befiltered=[]
for domain in monitorp:
    threading.Thread(target=scanDomain,
                    args=(domain,)).start()

In [19]:
befiltered=[]
for domain in HK_domain:
    threading.Thread(target=scanDomain,
                    args=(domain,)).start()

In [20]:
befiltered

['0.gravatar.com',
 '1.bp.blogspot.com',
 '163.com',
 '2.bp.blogspot.com',
 '3.bp.blogspot.com',
 '4.bp.blogspot.com',
 '4399.com',
 '56.com',
 '806.tianya.cn',
 'a-m-s.poco.cn',
 'a.cdn.intentmedia.net',
 'a.cntv.cn',
 'a.youdao.com',
 'a1331.phobos.apple.com',
 'a1946.phobos.apple.com',
 'a33.phobos.apple.com',
 'a4.att.hudong.com',
 'a505.phobos.apple.com',
 'a837.phobos.apple.com',
 'about.com',
 'ad.doubleclick.net',
 'ad.zanox.com',
 'addthis.com',
 'adm.shinobi.jp',
 'adclient-af.lp.uol.com.br',
 'ads.brazzers.com',
 'ads.contentabc.com',
 'ads2.contentabc.com',
 'adultfriendfinder.com',
 'adv.drtuber.com',
 'agt.p.360.cn',
 'ajax.googleapis.com',
 'alert.services.conduit.com',
 'alt1.gmail-smtp-in.l.google.com',
 'alt4.gmail-smtp-in.l.google.com',
 'ameblo.jp',
 'analytics.163.com',
 'api.addthis.com',
 'api.bing.com',
 'api.flickr.com',
 'api.g1.globo.com',
 'api.mixpanel.com',
 'api.simplyhired.com',
 'api.zanox.com',
 'app.feed.informer.com',
 'appimg.qq.com',
 'appldnld.app

In [21]:
pd.DataFrame(monitorp.T.index.drop(befiltered)).to_csv('malurl_filtered10000.txt', header=False, index=False)


In [20]:
a=pd.DataFrame()
a.to_csv

In [177]:
monitorp=monitorp.T.drop(befiltered)
monitorp=monitorp.T

In [178]:
directory=r'C:\Users\Pomodori\OneDrive\code\phoneDNS\hittedDomain_f\\'
if not os.path.exists(directory):
    os.makedirs(directory)
    
for i in monitorp.T:
    idx=monitorp.T[i][monitorp.T[i]!=0].index
    outname=str(i)
    f=open(directory+outname+'.txt','w')
    for i in idx:
        f.write(i+'\n')
    f.close()

##rank

In [46]:
urlcnt=monitorp.T.sum()
for i in range(len(urlcnt)):
    print urlcnt[urlcnt.argsort()].index[i],int(urlcnt[urlcnt.argsort()][i])

112.5.128.133 2
120.197.158.84 3
120.192.28.78 3
112.12.27.162 5
211.137.130.4 6
120.203.85.250 6
120.209.190.54 7
211.137.130.20 10
112.16.85.35 11
117.146.103.182 26
221.130.33.52 34
221.130.33.60 47
111.12.76.8 57
111.12.76.134 64
111.12.76.9 65
117.139.72.226 71
117.146.102.106 72
120.199.20.158 80
120.202.47.244 80
117.190.78.122 81
111.63.3.162 83
111.12.76.135 86
112.16.87.171 87
120.194.102.10 88
112.16.85.20 100
120.193.64.10 115
112.16.85.39 271
211.136.20.203 827
333.333.333.333 1072
120.199.132.94 5879
117.141.38.180 5881
117.141.38.170 5958
111.17.171.50 5974
111.47.61.2 6034
112.16.87.120 6034


### drop 0

In [58]:
monitorp=monitorp.T.drop(monitorp.sum().index[monitorp.sum()<=2])
monitorp=monitorp.drop(monitorp.T.sum().index[monitorp.T.sum()==0])

### analysis single col&row

In [47]:
divDomain=monitorp.sum()
a = divDomain # your array of numbers
plt.title('domain CDF')

In [49]:
divDNS=monitorp.T.sum()
a = divDNS # your array of numbers
plt.title('DNS CDF')

In [50]:
num_bins = 100
counts, bin_edges = np.histogram(a, bins=num_bins)
cdf = np.cumsum(counts)
plt.plot(bin_edges[:-1], cdf/float(cdf[-1]))
# plt.xlim([0, len(x)+1])
# plt.ylim([0, max(y_1+y_2) + 10])
plt.xlabel('x')
plt.ylabel('CDF')
#plt.legend(['sample 1', 'sample2'], loc='upper left')
plt.show()

In [8]:
from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

np.random.seed(42)

digits = load_digits()
#data = scale(digits.data)
data = scale(np.array(monitorp))

n_samples, n_features = data.shape

#n_digits = len(np.unique(digits.target))
n_digits=7
labels = digits.target

sample_size = 300

## clustering

In [14]:
print("n_digits: %d, \t n_samples %d, \t n_features %d"
      % (n_digits, n_samples, n_features))


print(79 * '_')
print('% 9s' % 'init'
      '    time  inertia    homo   compl  v-meas     ARI AMI  silhouette')


def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
#     print('% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
#           % (name, (time() - t0), estimator.inertia_,
#              metrics.homogeneity_score(labels, estimator.labels_),
#              metrics.completeness_score(labels, estimator.labels_),
#              metrics.v_measure_score(labels, estimator.labels_),
#              metrics.adjusted_rand_score(labels, estimator.labels_),
#              metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
#              metrics.silhouette_score(data, estimator.labels_,
#                                       metric='euclidean',
#                                       sample_size=sample_size)))

#bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=10),
#              name="k-means++", data=data)

# bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=10),
#               name="random", data=data)

# in this case the seeding of the centers is deterministic, hence we run the
# kmeans algorithm only once with n_init=1
pca = PCA(n_components=n_digits).fit(data)
bench_k_means(KMeans(init=pca.components_, n_clusters=n_digits, n_init=1),
              name="PCA-based",
              data=data)
print(79 * '_')

###############################################################################
# Visualize the results on PCA-reduced data

reduced_data = PCA(n_components=2).fit_transform(data)
kmeans = KMeans(init='k-means++', n_clusters=n_digits, n_init=10)
kmeans.fit(reduced_data)

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .02     # point in the mesh [x_min, m_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min() + 1, reduced_data[:, 0].max() - 1
y_min, y_max = reduced_data[:, 1].min() + 1, reduced_data[:, 1].max() - 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1)
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=2)
# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10)
plt.title('K-means clustering on the digits dataset (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()

n_digits: 7, 	 n_samples 3021, 	 n_features 33
_______________________________________________________________________________
init    time  inertia    homo   compl  v-meas     ARI AMI  silhouette
_______________________________________________________________________________


In [19]:
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])
np.c_
Z = Z.reshape(xx.shape)

In [31]:
result=kmeans.predict(reduced_data)
for r in range(n_digits):
    for i in range(len(result)):
        if result[i]==r:
            print monitorp.index[i]
    print 79*'_'

010707da1023.brightdog.net
011707db0108.kathell.com
012707da070b.kathell.com
111a.xdwan.cn
11814.domainname.filter.seedcornppc.com
120.nsb927.com
123.canaround.com
1312343955.3322.org
141607db013a.weirden.com
141607db0833.e-info-update.org
15012.filter.oridianppc.com
15083868.blog.hexun.com
1511.ic-live.com
151325.3322.org
154.nsvjn987.com
162707da103a.kathell.com
163mx00.mxmail.netease.com
168.xdwan.cn
1683.ezdrop.co.cc
17485.28309.filter.inforoar.com
175seer.tk
177.NS1631262.ORG
17f6c.makemegood24.com
18011.14099.filter2.conductsearch.com
181207da1127.brightdog.net
18487.11960.adintegrate.namiflow.com
187.ns792.com
18708.20426.filter.seedcornppc.com
190107db011a.weirden.com
1950519821165729028-a-1802744773732722657-s-sites.googlegroups.com
1986584077.mail.outlook.com
1hk-.wwwmediahosts.com
1wrench.com
1zdkmpg3tw.wwwmediahosts.com
2.nsb927.com
20ecb.cash-ddt.net
21.no-ip.info
212773.xz19.com
221007db0228.weirden.com
221107db012e.weirden.com
221907da092e.brightdog.net
23.WAP517.NET
232

## AgglomerativeClustering

In [96]:
# Authors: Gael Varoquaux
# License: BSD 3 clause (C) INRIA 2014

print(__doc__)
from time import time

import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt

from sklearn import manifold, datasets

nclass=15

digits = datasets.load_digits(n_class=nclass)

#X = digits.data
X = scale(np.array(monitorp.T))
#y = digits.target
y= [np.random.randint(nclass) for i in range(len(X))]
n_samples, n_features = X.shape

np.random.seed(0)

def nudge_images(X, y):
    # Having a larger dataset shows more clearly the behavior of the
    # methods, but we multiply the size of the dataset only by 2, as the
    # cost of the hierarchical clustering methods are strongly
    # super-linear in n_samples
    shift = lambda x: ndimage.shift(x.reshape((6,-1)),
                                  .3 * np.random.normal(size=2),
                                  mode='constant',
                                  ).ravel()
    X = np.concatenate([X, np.apply_along_axis(shift, 1, X)])
    Y = np.concatenate([y, y], axis=0)
    return X, Y


#X, y = nudge_images(X, y)


#----------------------------------------------------------------------
# Visualize the clustering
def plot_clustering(X_red, X, labels, title=None):
    x_min, x_max = np.min(X_red, axis=0), np.max(X_red, axis=0)
    X_red = (X_red - x_min) / (x_max - x_min)

    plt.figure(figsize=(6, 4))
    for i in range(X_red.shape[0]):
        plt.text(X_red[i, 0], X_red[i, 1], str(y[i]),
                 color=plt.cm.spectral(labels[i] / float(nclass)),
                 fontdict={'weight': 'bold', 'size': 9})

    plt.xticks([])
    plt.yticks([])
    if title is not None:
        plt.title(title, size=17)
    plt.axis('off')
    plt.tight_layout()

#----------------------------------------------------------------------
# 2D embedding of the digits dataset
print("Computing embedding")
X_red = manifold.SpectralEmbedding(n_components=2).fit_transform(X)
print("Done.")

Automatically created module for IPython interactive environment
Computing embedding
Done.


In [97]:
from sklearn.cluster import AgglomerativeClustering

for linkage in ('ward', 'average', 'complete'):
    if linkage!='average':
        continue
    clustering = AgglomerativeClustering(linkage=linkage, n_clusters=nclass)
    t0 = time()
    clustering.fit(X_red)
    print("%s : %.2fs" % (linkage, time() - t0))

    plot_clustering(X_red, X, clustering.labels_, "%s linkage" % linkage)


plt.show()

average : 0.00s


In [99]:
for j in range(nclass):
    for i in range(len(clustering.labels_)):
        if j==clustering.labels_[i]:
            print monitorp.T.index[i]
    print 7*'_'

111.12.76.9
111.63.3.162
117.190.78.122
221.130.33.60
_______
111.12.76.135
112.16.85.20
120.209.190.54
221.130.33.52
_______
111.17.171.50
117.141.38.170
117.141.38.180
_______
120.203.85.250
_______
120.202.47.244
_______
111.47.61.2
112.16.87.120
120.199.132.94
_______
111.12.76.134
112.16.87.171
117.139.72.226
117.146.103.182
120.193.64.10
_______
120.192.28.78
_______
112.12.27.162
_______
112.5.128.133
_______
120.197.158.84
_______
112.16.85.35
_______
111.12.76.8
112.16.85.39
117.146.102.106
120.194.102.10
211.137.130.20
_______
211.137.130.4
_______
120.199.20.158
_______


## system cluster

In [135]:
a=np.array(monitorp.T).tolist()
cluster=[]
for i in range(np.shape(a)[0]):
    cluster.append([i])
a=np.array(a)
centers=a.copy()
centers=centers.astype('float')
m,n=np.shape(a)

In [136]:
def mandist(vecA,vecB):
    return len(vecA)-((vecA==1)&(vecB==1)).sum()

In [137]:
for k in range(m-1):#m
    d=np.zeros((m-k,m-k))
    for i in range(m-k):
        for j in range(i+1,m-k):
            d[i,j]=mandist(centers[i],centers[j])
    nzInd=np.nonzero(d)                               #take care of it
    minInd=d[nzInd].argmin(0)
    mA=nzInd[0][minInd]
    mB=nzInd[1][minInd]
    cluster[mA].extend(cluster[mB])
    cluster.remove(cluster[mB])
    print cluster
    for i in range(m-k-1):
                                                  #cluster distance: core method
        centers[i]=np.mean(a[cluster[i]],axis=0)

[[0], [1], [2], [3], [4], [5, 11], [6], [7], [8], [9], [10], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]
[[0], [1], [2], [3], [4, 15], [5, 11], [6], [7], [8], [9], [10], [12], [13], [14], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]
[[0], [1], [2], [3], [4, 15], [5, 11, 24], [6], [7], [8], [9], [10], [12], [13], [14], [16], [17], [18], [19], [20], [21], [22], [23], [25], [26], [27], [28], [29], [30], [31], [32]]
[[0], [1], [2], [3], [4, 15, 16], [5, 11, 24], [6], [7], [8], [9], [10], [12], [13], [14], [17], [18], [19], [20], [21], [22], [23], [25], [26], [27], [28], [29], [30], [31], [32]]
[[0], [1], [2], [3], [4, 15, 16, 5, 11, 24], [6], [7], [8], [9], [10], [12], [13], [14], [17], [18], [19], [20], [21], [22], [23], [25], [26], [27], [28], [29], [30], [31], [32]]
[[0], [1], [2], [3], [4, 15, 16, 5, 11, 24], [6], [7], [8, 10], [9], [12], [13], [14]

## DBSCAN

In [9]:
# -*- coding: utf-8 -*-
"""
===================================
Demo of DBSCAN clustering algorithm
===================================

Finds core samples of high density and expands clusters from them.

"""
print(__doc__)

import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import scipy.spatial.distance as distance


Demo of DBSCAN clustering algorithm

Finds core samples of high density and expands clusters from them.




In [11]:
monitorp

dns                                                 111.12.76.134  \
domain                                                              
0-0-0-0-1-0-0-1-1-0-0-0-1-0-0-1-0-1-1-0-0-1-1-1...              1   
0-0-0-1-0-1-1-1-1-1-0-1-0-1-1-1-1-0-0-0-0-0-1-1...              0   
0-0-1-0-0-0-1-0-1-0-0-1-0-0-1-0-0-0-0-1-1-0-0-0...              0   
0-0-1-0-1-1-1-0-0-1-1-1-1-1-0-1-1-1-0-1-1-0-1-0...              0   
0-0-1-1-0-0-1-0-0-0-0-1-0-0-0-1-0-1-1-0-0-0-0-1...              0   
0-0-1-1-0-1-1-0-1-1-0-1-0-1-1-0-0-0-0-1-0-1-0-1...              0   
0-0-1-1-1-0-1-1-1-0-0-1-1-0-1-0-1-1-0-0-1-0-0-1...              0   
0-1-0-0-0-0-0-0-0-1-0-1-1-1-1-1-0-1-1-1-1-1-0-1...              0   
0-1-0-0-0-1-1-0-0-1-0-1-0-1-0-1-0-1-0-1-1-1-1-0...              0   
0-1-0-0-0-1-1-1-1-0-0-0-0-1-1-0-1-0-0-0-1-0-1-1...              0   
0-1-0-0-1-0-0-0-1-0-1-1-0-1-1-1-1-0-1-1-1-0-0-0...              0   
0-1-4-i-g-2-d-v-y-w-z-3-0-4-g-9-i-0-0-4-5-g-f-7...              0   
0-1-h-2-5-d-8-l-z-b-0-2-j-n-3-4-d-6-9-3-8-a-d-9...              0   
0-2-1-d-6-b-4-2-7-3-9-i-8-7-8-c-e-e-c-m-6-x-a-3...              0   
0-3-3-2-8-7-6-3-6-2-3-3-8-1-5-3-0-1-0-3-8-4-4-8...              0   
0-4-5-4-7-0-1-6-4-3-1-6-8-4-0-6-3-4-4-1-2-0-5-1...              0   
0-4-8-0-6-9-b-8-d-g-0-6-2-p-5-f-y-6-4-7-x-b-7-6...              0   
0-4-v-0-3-5-5-n-r-w-k-i-7-g-3-k-k-e-9-2-l-9-7-4...              0   
0-5-4-y-7-6-4-6-0-i-q-8-a-m-q-6-z-4-2-r-3-4-8-x...              0   
0-6-4-4-0-0-0-0-0-0-.0-0-0-0-0-0-0-0-0-0-0-0-0-...              0   
0-6-s-k-j-4-g-6-v-x-e-0-5-0-h-0-a-1-0-5-y-8-l-4...              0   
0-i-x-a-j-o-e-8-3-q-5-9-s-8-5-7-p-3-9-p-1-3-i-a...              0   
0-kkjda-l.hdmediastore.com                                      0   
0-nethosting.com                                                0   
0-o-r-r-s-g-k-u-f-a-z-q-g-3-7-t-h-2-t-4-p-c-g-4...              0   
0-ps9al6c.remindmeroster.com                                    0   
0.blogger.gmodules.com                                          0   
0.byinter.net                                                   0   
0.gravatar.com                                                  1   
0.mx.softhome.net                                               0   
...                                                           ...   
yandex.ua                                                       0   
yaplog.jp                                                       0   
yeskyafp.allyes.com                                             0   
yilong9418.blog.163.com                                         0   
ylog.hiido.com                                                  0   
ymail.com                                                       0   
yngogogo.3322.org                                               0   
yourjavascript.com                                              0   
youtube.com                                                     0   
ys-i.ys168.com                                                  1   
ys6067.91mt.com                                                 0   
yun.babytree.com                                                0   
yunfile.com                                                     0   
yunxiao120.blog.163.com                                         0   
yy.com                                                          0   
z.abang.com                                                     0   
z.rising.com.cn                                                 0   
zandaa.myvnc.com                                                0   
zappos.com                                                      0   
zc.antivirusbar.org                                             0   
zdnet.com                                                       1   
zhengshi1.3322.org                                              0   
zone124.hotwords.com.br                                         0   
zone81.hotwords.com.br                                          0   
zopim.com                                                       1  

In [20]:
data=np.array(monitorp).astype(bool)
precomputedData=[]
for i in data:
    rows=[]
    for j in data:
        
        rows.append(distance.jaccard(i,j))
    precomputedData.append(rows)
precomputedData=np.array(precomputedData)
print precomputedData

[[ 0.          0.2         0.4        ...,  0.81818182  0.57894737  0.7       ]
 [ 0.2         0.          0.25       ...,  0.77777778  0.68421053  0.625     ]
 [ 0.4         0.25        0.         ...,  0.71428571  0.72222222  0.5       ]
 ..., 
 [ 0.81818182  0.77777778  0.71428571 ...,  0.          0.82352941  0.5       ]
 [ 0.57894737  0.68421053  0.72222222 ...,  0.82352941  0.          0.82352941]
 [ 0.7         0.625       0.5        ...,  0.5         0.82352941  0.        ]]


In [21]:

eps=np.arange(0.05,0.5,0.05)
spl=np.arange(1,8,1)
eps,spl=np.meshgrid(eps,spl)
eps=eps.ravel()
spl=spl.ravel()

In [22]:
avaiepsandsamples=[]
for i in range(len(eps)):
    # Compute DBSCAN
    db = DBSCAN(eps=eps[i],
                metric='precomputed',
                min_samples=spl[i]).fit(precomputedData)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    
    if n_clusters_<=2:
        pass
    else:
        avaiepsandsamples.append([int(eps[i]*1000)/1000.0,spl[i],n_clusters_])

In [23]:
print avaiepsandsamples

[[0.05, 1, 1348], [0.1, 1, 1346], [0.15, 1, 1285], [0.2, 1, 933], [0.25, 1, 448], [0.3, 1, 410], [0.35, 1, 350], [0.4, 1, 178], [0.45, 1, 149], [0.05, 2, 292], [0.1, 2, 294], [0.15, 2, 279], [0.2, 2, 214], [0.25, 2, 47], [0.3, 2, 47], [0.35, 2, 42], [0.4, 2, 22], [0.45, 2, 18], [0.05, 3, 161], [0.1, 3, 161], [0.15, 3, 154], [0.2, 3, 121], [0.25, 3, 14], [0.3, 3, 14], [0.35, 3, 11], [0.4, 3, 6], [0.45, 3, 6], [0.05, 4, 119], [0.1, 4, 119], [0.15, 4, 111], [0.2, 4, 98], [0.25, 4, 4], [0.3, 4, 4], [0.4, 4, 3], [0.45, 4, 3], [0.05, 5, 95], [0.1, 5, 95], [0.15, 5, 89], [0.2, 5, 83], [0.25, 5, 4], [0.3, 5, 5], [0.05, 6, 84], [0.1, 6, 84], [0.15, 6, 80], [0.2, 6, 72], [0.25, 6, 3], [0.3, 6, 3], [0.05, 7, 76], [0.1, 7, 76], [0.15, 7, 72], [0.2, 7, 67]]


In [37]:
eps=0.25
min_samples=3
# Compute DBSCAN
db = DBSCAN(eps=eps,
            metric='precomputed',
            min_samples=min_samples).fit(X=precomputedData)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print n_clusters_
labelsgb=monitorp.T.sum().groupby(labels)

14


In [38]:
outname=str(eps)+'_'+str(min_samples)
directory=r'C:\Users\Pomodori\OneDrive\code\phoneDNS\DBSCAN_'+outname+'\\'
if not os.path.exists(directory):
    os.makedirs(directory)
labelsgb_mean=labelsgb.mean()
labelsgb_mean.to_csv(directory+'meanhitrate.txt')
for i in set(labels):
    f=open(directory+str(i)+'.txt','w')
for i in range(len(monitorp.index)):
    j=labels[i]
    f=open(directory+str(j)+'.txt','a')
    f.write(monitorp.index[i]+'\n')
for i in set(labels):
    f=open(directory+str(i)+'.txt','r')
    f.close()

In [ ]:
print('Estimated number of clusters: %d' % n_clusters_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

##############################################################################

In [138]:
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = 'k'

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()


Demo of DBSCAN clustering algorithm

Finds core samples of high density and expands clusters from them.


Estimated number of clusters: 3
Homogeneity: 0.953
Completeness: 0.883
V-measure: 0.917
Adjusted Rand Index: 0.952
Adjusted Mutual Information: 0.883
Silhouette Coefficient: 0.626
